Imports

In [78]:
import csv
import pandas as pd
from datetime import datetime
from datetime import timedelta
import os

In [79]:
def secondsToUTC (dataset):
    #1989-12-31 00:00:00
    zero = datetime(1989,12,31,0,0,0)
    #print (datetime.strftime(zero,"%b %d %Y %H:%M:%S"))
    new_dataset = []
    for d in dataset:
        n = str(zero + timedelta(seconds=d))
        new_dataset.append(n)
    return new_dataset

In [80]:
def prepareCSV (filepath, file, filetype):
    #Read csv
    df = pd.read_csv(filepath+file+filetype)
    
    #filter for local number == 7 and type == Date
    df_filtered = df.loc[df['Local Number'] == 7]
    df_filtered = df_filtered.loc[df_filtered['Type'] == 'Data']
    
    #convert position and time
    df_filtered['Value 2'] = df_filtered['Value 2']* 180/(2**31)
    df_filtered['Units 2'] = "degrees"

    df_filtered['Value 3'] = df_filtered['Value 3']* 180/(2**31)
    df_filtered['Units 3'] = "degrees"

    df_filtered['Value 1'] = secondsToUTC(df_filtered['Value 1'])
    df_filtered['Units 1'] = "UTC"
    
    # drop unnecessary data
    toBeDropped = []
    counter = 10
    field_str = 'Field '
    value_str = 'Value '
    units_str = 'Units '

    while counter < 24:
        toBeDropped.append(field_str+str(counter))
        toBeDropped.append(value_str+str(counter))
        toBeDropped.append(units_str+str(counter))
        counter = counter + 1

    toBeDropped.append("Type")
    toBeDropped.append("Local Number")
    toBeDropped.append("Message")
    toBeDropped.append("Unnamed: 72")
    
    df_filtered = df_filtered.drop(columns=toBeDropped)
    
    # write csv
    df_filtered.to_csv(filepath+file+'_prepared'+filetype)

In [81]:
file = '921B4648'
filepath = '/home/julio/strides/data/fit2csv/'
filetype = '.csv'

Get file names in directory


In [84]:
files = []
for file in os.listdir(filepath):
    if file.endswith(".csv") and not file.endswith("prepared.csv"):
        files.append(file)
print(files)

['913F2746.csv', '91SI5602.csv', '916A2805.csv', '911E0950.csv', '911D3633.csv', '91VI5920.csv', '914H2817.csv', '921B4648.csv', '91U70725.csv', '91VI2925.csv', '915F0120.csv', '911B3000.csv', '919H2158.csv', '911C4903.csv']


prepare files

In [83]:
for file in files:
    prepareCSV (filepath, file, filetype)

FileNotFoundError: [Errno 2] File b'/home/julio/strides/data/fit2csv/913F2746.csv.csv' does not exist: b'/home/julio/strides/data/fit2csv/913F2746.csv.csv'

,Field 1,Value 1,Units 1,Field 2,Value 2,Units 2,Field 3,Value 3,Units 3,Field 4,...,Units 6,Field 7,Value 7,Units 7,Field 8,Value 8,Units 8,Field 9,Value 9,Units 9
19,timestamp,2019-02-01 10:46:48,UTC,position_lat,48.139598,degrees,position_long,11.589160,degrees,distance,...,NaN,heart_rate,90.0,bpm,cadence,79.0,rpm,enhanced_speed,2.682,m/s
20,timestamp,2019-02-01 10:46:49,UTC,position_lat,48.139640,degrees,position_long,11.589109,degrees,distance,...,NaN,heart_rate,92.0,bpm,cadence,79.0,rpm,enhanced_speed,3.348,m/s
23,timestamp,2019-02-01 10:46:50,UTC,position_lat,48.139677,degrees,position_long,11.589135,degrees,distance,...,NaN,heart_rate,95.0,bpm,cadence,0.0,rpm,enhanced_speed,3.583,m/s
24,timestamp,2019-02-01 10:46:51,UTC,position_lat,48.139669,degrees,position_long,11.589165,degrees,distance,...,NaN,heart_rate,97.0,bpm,cadence,0.0,rpm,enhanced_speed,3.337,m/s
25,timestamp,2019-02-01 10:46:52,UTC,position_lat,48.139700,degrees,position_long,11.589201,degrees,distance,...,NaN,heart_rate,100.0,bpm,cadence,0.0,rpm,enhanced_speed,3.542,m/s


save